In [240]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import json
import os

pd.set_option('display.max_rows', 200)

In [241]:
%load_ext dotenv
%dotenv ../.env

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [242]:
con = create_engine(os.getenv('EDM_DATA'))

In [243]:
# Build funtion to get the domains associated with a specific field
def get_domain(field, domain_path):
    with open(domain_path, "r") as domain:
        domain = json.load(domain)
    records = [i for i in domain['fields'] if i['field'] == field]
    if len(records) == 0: 
        print(f'cannot find field: {field} in domain table')
    else: 
        return records[0]['domain']

In [244]:
# Example
proxcode_domain = get_domain('proxcode', 'pluto_domains.json')
print(proxcode_domain)

{'0': 'Not Available', '1': 'Detached', '2': 'Semi-Attached', '3': 'Attached'}


In [245]:
# Example
proxcode_domain.keys()

dict_keys(['0', '1', '2', '3'])

In [246]:
# Specify latest PLUTO version
pluto = 'dcp_pluto."19v2_wo_corrections"'
# Specify previous PLUTO version or the one you want to compare to
plutoprev = 'dcp_pluto."19v1"'

### Value check for the thirteen fields with expected domain values

##### includes summary statistics of unique value comparisions for the two PLUTO version, and lists of values that do not agree w/ domain list

### Zoning districts

In [247]:
sql = f'''
    WITH newcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {pluto})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist),
    prevcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {plutoprev})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist)
    SELECT a.zonedist, b.zonedist as zonedistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.zonedist = b.zonedist
    ORDER BY diff DESC;
'''

In [248]:
df = pd.read_sql(sql=sql, con=con)

In [249]:
df[df['diff'] != 0]

,zonedist,zonedistprev,countnew,countprev,diff
0,None,None,NaN,2560881.0,NaN
1,None,None,2567266.0,NaN,NaN
2,R2X,R2X,1385.0,1007.0,378.0
3,R6A,R6A,16168.0,16136.0,32.0
4,R3-1,R3-1,51487.0,51470.0,17.0
5,R3-2,R3-2,63432.0,63415.0,17.0
6,R3X,R3X,51695.0,51678.0,17.0
7,R4,R4,75988.0,75975.0,13.0
8,R4-1,R4-1,50679.0,50669.0,10.0
9,R3A,R3A,54847.0,54837.0,10.0


### Commercial overlay

In [250]:
sql = f'''
    WITH newcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {pluto}
    UNION ALL
    SELECT overlay2 as overlay FROM {pluto}
    )
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay),
    prevcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {plutoprev}
    UNION ALL
    SELECT overlay2 as overlay FROM {plutoprev})
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay)
    SELECT a.overlay, b.overlay as overlayprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.overlay = b.overlay
    ORDER BY a.overlay;
'''

In [251]:
df = pd.read_sql(sql=sql, con=con)

In [252]:
df

,overlay,overlayprev,countnew,countprev,diff
0,C1-1,C1-1,1590.0,1587.0,3.0
1,C1-2,C1-2,11031.0,11031.0,0.0
2,C1-3,C1-3,13805.0,13790.0,15.0
3,C1-4,C1-4,8093.0,8097.0,-4.0
4,C1-5,C1-5,2541.0,2547.0,-6.0
5,C2-1,C2-1,1682.0,1684.0,-2.0
6,C2-2,C2-2,6024.0,6023.0,1.0
7,C2-3,C2-3,11946.0,11952.0,-6.0
8,C2-4,C2-4,16728.0,16727.0,1.0
9,C2-5,C2-5,1523.0,1523.0,0.0


### Special purpose district

In [253]:
sql = f'''
    WITH newcount as (
    WITH sdall as (
    SELECT spdist1 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist2 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist3 as spdist FROM {pluto}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist),
    prevcount as (
     WITH sdall as (
    SELECT spdist1 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist2 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist3 as spdist FROM {plutoprev}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist)
    SELECT a.spdist, b.spdist as spdistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.spdist = b.spdist
    ORDER BY diff DESC;
'''

In [254]:
df = pd.read_sql(sql=sql, con=con)

In [255]:
df[df['diff'] != 0]

,spdist,spdistprev,countnew,countprev,diff
0,None,None,NaN,2476583.0,NaN
1,None,None,2481371.0,NaN,NaN
2,MX-18,None,1.0,NaN,NaN
3,SRD,SRD,43902.0,43889.0,13.0
4,HS,HS,6167.0,6165.0,2.0
5,HY,HY,290.0,289.0,1.0
6,MX-1,MX-1,238.0,237.0,1.0
7,SG,SG,228.0,227.0,1.0
8,MiD,MiD,1486.0,1485.0,1.0
71,PI,PI,451.0,452.0,-1.0


In [256]:
#Check spdist domain
unique_spdist = df.spdist.tolist()
spdist_domain = list(get_domain('spdist1', 'pluto_domains.json').keys())

In [257]:
# in domain not in pluto:
[i for i in spdist_domain if i not in unique_spdist]

['MX-3']

In [258]:
# in pluto not in domain:
[i for i in unique_spdist if i not in spdist_domain]

[None, None, 'MX-18', 'BSC', 'SW', 'MX-20', 'MX-16/EC-6', 'MX-16/EC-5']

### Limited height

In [259]:
sql = f'''
    WITH newcount as (
    SELECT ltdheight, count(*)
    FROM {pluto}
    GROUP BY ltdheight),
    prevcount as (
    SELECT ltdheight, count(*)
    FROM {plutoprev}
    GROUP BY ltdheight)
    SELECT a.ltdheight, b.ltdheight as ltdheightprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ltdheight = b.ltdheight
    ORDER BY a.ltdheight;
'''

In [260]:
df = pd.read_sql(sql=sql, con=con)

In [261]:
df

,ltdheight,ltdheightprev,countnew,countprev,diff
0,LH-1,LH-1,2239.0,2239.0,0.0
1,LH-1A,LH-1A,797.0,797.0,0.0
2,None,None,858017.0,NaN,NaN
3,None,None,NaN,856423.0,NaN


In [262]:
#Check ltdheight domain
unique_ltdheight = df.ltdheight.tolist()
ltdheight_domain = list(get_domain('ltdheight', 'pluto_domains.json').keys())

In [263]:
# in domain not in pluto:
[i for i in ltdheight_domain if i not in unique_ltdheight]

['LH-2', 'LH-3']

In [264]:
# in pluto not in domain:
[i for i in unique_ltdheight if i not in ltdheight_domain]

[None, None]

### Building class

In [265]:
sql = f'''
    WITH newcount as (
    SELECT bldgclass, count(*)
    FROM {pluto}
    GROUP BY bldgclass),
    prevcount as (
    SELECT bldgclass, count(*)
    FROM {plutoprev}
    GROUP BY bldgclass)
    SELECT a.bldgclass, b.bldgclass as bldgclassprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bldgclass = b.bldgclass
    ORDER BY diff DESC;
'''

In [266]:
df = pd.read_sql(sql=sql, con=con)

In [267]:
df[df['diff'] != 0]

,bldgclass,bldgclassprev,countnew,countprev,diff
0,None,RT,NaN,8.0,NaN
1,None,L1,NaN,73.0,NaN
2,None,None,429.0,NaN,NaN
3,None,None,NaN,473.0,NaN
4,A5,A5,102449.0,100982.0,1467.0
5,R4,R4,2164.0,1341.0,823.0
6,R2,R2,953.0,554.0,399.0
7,K4,K4,6369.0,6094.0,275.0
8,B2,B2,79407.0,79163.0,244.0
9,R1,R1,1852.0,1711.0,141.0


In [268]:
#Check bldgclass domain
unique_bldgclass = df.bldgclass.tolist()
bldgclass_domain = list(get_domain('bldgclass', 'pluto_domains.json').keys())

In [269]:
# in domain not in pluto:
[i for i in bldgclass_domain if i not in unique_bldgclass]

['A',
 'B',
 'C',
 'D',
 'E',
 'E3',
 'E4',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'L1',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'R7',
 'RT',
 'S',
 'T',
 'U',
 'U3',
 'V',
 'W',
 'Y',
 'Z',
 'Z6']

In [270]:
# in pluto not in domain:
[i for i in unique_bldgclass if i not in bldgclass_domain]

[None, None, None, None]

### Land use

In [271]:
sql = f'''
    WITH newcount as (
    SELECT landuse, count(*)
    FROM {pluto}
    GROUP BY landuse),
    prevcount as (
    SELECT landuse, count(*)
    FROM {plutoprev}
    GROUP BY landuse)
    SELECT a.landuse, b.landuse as landuseprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.landuse = b.landuse
    ORDER BY a.landuse;
'''

In [272]:
df = pd.read_sql(sql=sql, con=con)

In [273]:
df[df['diff'] != 0]

,landuse,landuseprev,countnew,countprev,diff
0,01,01,565492.0,565356.0,136.0
1,02,02,131989.0,131378.0,611.0
2,03,03,13690.0,12846.0,844.0
3,04,04,54964.0,54508.0,456.0
4,05,05,21609.0,21482.0,127.0
5,06,06,9954.0,10497.0,-543.0
6,07,07,6315.0,6335.0,-20.0
7,08,08,12192.0,12177.0,15.0
8,09,09,4036.0,4017.0,19.0
9,10,10,10260.0,10224.0,36.0


In [274]:
#Check landuse domain
unique_landuse = df.landuse.tolist()
landuse_domain = list(get_domain('landuse', 'pluto_domains.json').keys())

In [275]:
# in domain not in pluto:
[i for i in landuse_domain if i not in unique_landuse]

[]

In [276]:
# in pluto not in domain:
[i for i in unique_landuse if i not in landuse_domain]

[None, None]

### Owner type

In [277]:
sql = f'''
    WITH newcount as (
    SELECT ownertype, count(*)
    FROM {pluto}
    GROUP BY ownertype),
    prevcount as (
    SELECT ownertype, count(*)
    FROM {plutoprev}
    GROUP BY ownertype)
    SELECT a.ownertype, b.ownertype as ownertypeprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ownertype = b.ownertype
    ORDER BY a.ownertype;
'''

In [278]:
df = pd.read_sql(sql=sql, con=con)

In [279]:
df

,ownertype,ownertypeprev,countnew,countprev,diff
0,C,C,11654.0,11654.0,0.0
1,M,M,80.0,80.0,0.0
2,O,O,1373.0,1373.0,0.0
3,P,P,522.0,522.0,0.0
4,X,X,19988.0,17414.0,2574.0
5,None,None,827436.0,NaN,NaN
6,None,None,NaN,828416.0,NaN


In [280]:
#Check ownertype domain
unique_ownertype = df.ownertype.tolist()
ownertype_domain = list(get_domain('ownertype', 'pluto_domains.json').keys())

In [281]:
# in domain not in pluto:
[i for i in ownertype_domain if i not in unique_ownertype]

['NaN']

In [282]:
# in pluto not in domain:
[i for i in unique_ownertype if i not in ownertype_domain]

[None, None]

### Area source

In [283]:
sql = f'''
    WITH newcount as (
    SELECT areasource, count(*)
    FROM {pluto}
    GROUP BY areasource),
    prevcount as (
    SELECT areasource, count(*)
    FROM {plutoprev}
    GROUP BY areasource)
    SELECT a.areasource, b.areasource as areasourceprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.areasource = b.areasource
    ORDER BY a.areasource;
'''

In [284]:
df = pd.read_sql(sql=sql, con=con)

In [285]:
df

,areasource,areasourceprev,countnew,countprev,diff
0,0,0,163.0,136.0,27.0
1,2,2,816705.0,813263.0,3442.0
2,4,4,259.0,182.0,77.0
3,5,5,119.0,63.0,56.0
4,7,7,43378.0,45342.0,-1964.0
5,None,None,429.0,NaN,NaN
6,None,None,NaN,473.0,NaN


In [286]:
#Check areasource domain
unique_areasource = df.areasource.tolist()
areasource_domain = list(get_domain('areasource', 'pluto_domains.json').keys())

In [287]:
# in domain not in pluto:
[i for i in areasource_domain if i not in unique_areasource]

[]

In [288]:
# in pluto not in domain:
[i for i in unique_areasource if i not in areasource_domain]

[None, None]

### Extension

In [289]:
sql = f'''
    WITH newcount as (
    SELECT ext, count(*)
    FROM {pluto}
    GROUP BY ext),
    prevcount as (
    SELECT ext, count(*)
    FROM {plutoprev}
    GROUP BY ext)
    SELECT a.ext, b.ext as extprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ext = b.ext
    ORDER BY a.ext;
'''

In [290]:
df = pd.read_sql(sql=sql, con=con)

In [291]:
df

,ext,extprev,countnew,countprev,diff
0,E,E,48333.0,48335.0,-2.0
1,EG,EG,36692.0,36709.0,-17.0
2,G,G,264441.0,264413.0,28.0
3,N,N,451458.0,451333.0,125.0
4,None,None,60129.0,NaN,NaN
5,None,None,NaN,58669.0,NaN


In [292]:
#Check extension domain
unique_extension = df.ext.tolist()
extension_domain = list(get_domain('ext', 'pluto_domains.json').keys())

In [293]:
# in domain not in pluto:
[i for i in extension_domain if i not in unique_extension]

[]

In [294]:
# in pluto not in domain:
[i for i in unique_extension if i not in extension_domain]

['N', None, None]

### Proxy code

In [295]:
sql = f'''
    WITH newcount as (
    SELECT proxcode, count(*)
    FROM {pluto}
    GROUP BY proxcode),
    prevcount as (
    SELECT proxcode, count(*)
    FROM {plutoprev}
    GROUP BY proxcode)
    SELECT a.proxcode, b.proxcode as proxcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.proxcode = b.proxcode
    ORDER BY a.proxcode;
'''

In [296]:
df = pd.read_sql(sql=sql, con=con)

In [297]:
df

,proxcode,proxcodeprev,countnew,countprev,diff
0,0,0,145130.0,144832.0,298.0
1,1,1,313978.0,313731.0,247.0
2,2,2,200438.0,199995.0,443.0
3,3,3,201078.0,200428.0,650.0
4,None,None,429.0,NaN,NaN
5,None,None,NaN,473.0,NaN


In [298]:
#Check proxcode domain
unique_proxcode = df.proxcode.tolist()
proxcode_domain = list(get_domain('proxcode', 'pluto_domains.json').keys())

In [299]:
# in domain not in pluto:
[i for i in proxcode_domain if i not in unique_proxcode]

[]

In [300]:
# in pluto not in domain:
[i for i in unique_proxcode if i not in proxcode_domain]

[None, None]

### Lot type

In [301]:
sql = f'''
    WITH newcount as (
    SELECT lottype, count(*)
    FROM {pluto}
    GROUP BY lottype),
    prevcount as (
    SELECT lottype, count(*)
    FROM {plutoprev}
    GROUP BY lottype)
    SELECT a.lottype, b.lottype as lottypeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.lottype = b.lottype
    ORDER BY a.lottype;
'''

In [302]:
df = pd.read_sql(sql=sql, con=con)

In [303]:
df

,lottype,lottypeprev,countnew,countprev,diff
0,0,0,10419.0,8534.0,1885.0
1,1,1,1456.0,1456.0,0.0
2,2,2,1826.0,1827.0,-1.0
3,3,3,100645.0,100674.0,-29.0
4,4,4,3411.0,3438.0,-27.0
5,5,5,737857.0,738045.0,-188.0
6,6,6,2229.0,2228.0,1.0
7,7,7,63.0,64.0,-1.0
8,8,8,2528.0,2530.0,-2.0
9,9,9,190.0,190.0,0.0


In [304]:
#Check lottype domain
unique_lottype = df.lottype.tolist()
lottype_domain = list(get_domain('lottype', 'pluto_domains.json').keys())

In [305]:
# in domain not in pluto:
[i for i in lottype_domain if i not in unique_lottype]

[]

In [306]:
# in pluto not in domain:
[i for i in unique_lottype if i not in lottype_domain]

[None, None]

### Basement code

In [307]:
sql = f'''
    WITH newcount as (
    SELECT bsmtcode, count(*)
    FROM {pluto}
    GROUP BY bsmtcode),
    prevcount as (
    SELECT bsmtcode, count(*)
    FROM {plutoprev}
    GROUP BY bsmtcode)
    SELECT a.bsmtcode, b.bsmtcode as bsmtcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bsmtcode = b.bsmtcode
    ORDER BY a.bsmtcode;
'''

In [308]:
df = pd.read_sql(sql=sql, con=con)

In [309]:
df

,bsmtcode,bsmtcodeprev,countnew,countprev,diff
0,0,0,74405.0,74508.0,-103.0
1,1,1,153648.0,153697.0,-49.0
2,2,2,495983.0,495979.0,4.0
3,3,3,1511.0,1498.0,13.0
4,4,4,4723.0,4705.0,18.0
5,5,5,130354.0,128599.0,1755.0
6,None,None,429.0,NaN,NaN
7,None,None,NaN,473.0,NaN


In [310]:
#Check bsmtcode domain
unique_bsmtcode = df.bsmtcode.tolist()
bsmtcode_domain = list(get_domain('bsmtcode', 'pluto_domains.json').keys())

In [311]:
# in domain not in pluto:
[i for i in bsmtcode_domain if i not in unique_bsmtcode]

[]

In [312]:
# in pluto not in domain:
[i for i in unique_bsmtcode if i not in bsmtcode_domain]

[None, None]

### PLUTO map id

In [313]:
sql = f'''
    WITH newcount as (
    SELECT plutomapid, count(*)
    FROM {pluto}
    GROUP BY plutomapid),
    prevcount as (
    SELECT plutomapid, count(*)
    FROM {plutoprev}
    GROUP BY plutomapid)
    SELECT a.plutomapid, b.plutomapid as plutomapidprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.plutomapid = b.plutomapid
    ORDER BY a.plutomapid;
'''

In [314]:
df = pd.read_sql(sql=sql, con=con)

In [315]:
df

,plutomapid,plutomapidprev,countnew,countprev,diff
0,1,1,856853,856861,-8
1,2,2,3561,1915,1646
2,3,3,425,469,-44
3,4,4,210,210,0
4,5,5,4,4,0


In [316]:
#Check plutomapid domain
unique_plutomapid = df.plutomapid.tolist()
plutomapid_domain = list(get_domain('plutomapid', 'pluto_domains.json').keys())

In [317]:
# in domain not in pluto:
[i for i in plutomapid_domain if i not in unique_plutomapid]

[]

In [318]:
# in pluto not in domain:
[i for i in unique_plutomapid if i not in plutomapid_domain]

[]

#### done